<a href="https://colab.research.google.com/github/yorkjong/news-digest/blob/main/notebooks/news_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這支 colab notebook 主要功能是
- 剪輯 [news-digest](https://news-digest.vercel.app/) 網站當天（或選取日期）的新聞。
- 只能根據新聞分類 (catetories) 剪輯。

執行步驟如下：
1. 按下 Install 下那個 code cell 前面的 ► 按鈕來安裝用到的模組
2. 根據喜好，填表執行後面的 Jupyter Widgets UI 或 Colab Forms

需要更細的檢索查詢，有如下兩個選擇
- 直接在 [news-digest 網站](https://news-digest.vercel.app/)上搜尋或點選 hash-tags
- 或者點擊 [news_query.ipynb](https://colab.research.google.com/github/yorkjong/news-digest/blob/main/notebooks/news_query.ipynb) 執行多條件檢索


### Install

In [8]:
import os

fns = ['clip.py', 'op.py']

for fn in fns:
    if os.path.exists(fn):
        os.remove(fn)
for fn in fns:
    url = f'https://raw.githubusercontent.com/yorkjong/news-digest/main/src/{fn}'
    !wget $url

import clip

#-------------------------------------------------------------------------------

import ipywidgets as widgets

def create_ui():
    fns = sorted(clip.get_recent_journal_filenames(14), reverse=True)
    options = [f[:-3] for f in fns]
    dates = widgets.Dropdown(
        options=options,
        value=options[0],
        description='Date',
        disabled=False,
    )

    content = clip.get_latest_journal()
    options = clip.get_categories(content)
    categories = widgets.SelectMultiple(
        options=options,
        #value=[options[0]],
        rows=len(options),
        description='Categories',
        disabled=False,
    )

    with_hashtags = widgets.Checkbox(
        value=True,
        description='show hashtags',
        disabled=False,
    )

    with_headings = widgets.Checkbox(
        value=True,
        description='show headings',
        disabled=False,
    )

    vbox1 = widgets.VBox([dates, categories])
    vbox2 = widgets.VBox([with_hashtags, with_headings])
    output = widgets.Output()
    display(widgets.HBox([vbox1, vbox2]), output)

    def on_date_change(change):
        nonlocal content
        content = clip.get_journal(f'{dates.value}.md')
        categories.options = clip.get_categories(content)

    def on_value_change(change):
        lines = clip.get_lines_of_categories(categories.value, content, with_hashtags.value, with_headings.value)
        with output:
            #print(change['new'])
            output.clear_output()
            print('\n'.join(lines))

    dates.observe(on_date_change, names='value')
    categories.observe(on_value_change, names='value')
    with_hashtags.observe(on_value_change, names='value')
    with_headings.observe(on_value_change, names='value')

--2024-08-14 16:57:32--  https://raw.githubusercontent.com/yorkjong/news-digest/main/src/clip.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8936 (8.7K) [text/plain]
Saving to: ‘clip.py’

clip.py             100%[===================>]   8.73K  --.-KB/s    in 0s      

2024-08-14 16:57:32 (57.3 MB/s) - ‘clip.py’ saved [8936/8936]

--2024-08-14 16:57:32--  https://raw.githubusercontent.com/yorkjong/news-digest/main/src/op.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5171 (5.0K) [text/plain]
Saving to: ‘op.py’

op.py       

### Select categories with Jupyter Widgets
- 按下面那個 code cell 前面的 ► 按鈕，以列出完整的分類(Catetories)選單
- macOS 下操作可按Cmd後用滑鼠複選（Windows 系統則按下Ctrl鍵後以滑鼠複選）
  - 全選可按 Cmd+A (macOS) 或者 Ctrl+A (Windows)

In [9]:
create_ui()

Output()

### Select Categories with the Colab Forms

In [10]:
#@title Single Selection { run: "auto", display-mode: "form" }
category = "IT" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "IT"] {allow-input: true}
show_hashtags = True #@param {type:"boolean"}
show_headings = True #@param {type:"boolean"}

content = clip.get_latest_journal()
lines = clip.get_lines_of_categories([category], content, show_hashtags, show_headings)
text = '\n'.join(lines)
print(text)

### IT
- [採用新架構的 AI 模型「Falcon Mamba 7B」，表現直逼各大 Transformer 模型！](https://www.inside.com.tw/article/35900-falcon-mamba-7b) #Mamba #AI #GAI
- [Google發表挑戰OpenAI進階語音模式的Gemini Live](https://www.ithome.com.tw/news/164458) #GOOG #GeminiLive
- [谷歌向蘋果三星開火！深夜甩出四款AI手機，一鍵召喚Gemini，安卓全面AI化](https://news.cnyes.com/news/id/5683213) #GOOG
- [《Boss AI》超強AI回信、寫信和自動校正重寫輔助神器APP](https://mrmad.com.tw/boss-ai) #AI
- [NVIDIA中國特供晶片太貴 新創公司轉向這張顯示卡](https://news.xfastest.com/nvidia/143572/nvidia-china/) #NVDA #RTX4090
- [Appier公布第2季財報 營收創新高、營業利益年增4.4倍](https://udn.com/news/story/7241/8161495) #Appier #AI #TW



In [11]:
#@title Multiple Selections { run: "auto", display-mode: "form" }
first_category = "Tesla & SpaceX; Vehicle" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "IT"] {allow-input: true}
last_category = "Finance" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "IT"] {allow-input: true}
show_hashtags = False #@param {type:"boolean"}
show_headings = False #@param {type:"boolean"}

content = clip.get_latest_journal()
categories = clip.get_categories(content)
categories = clip.get_sublist(categories, first_category, last_category)
lines = clip.get_lines_of_categories(categories, content, show_hashtags, show_headings)
text = '\n'.join(lines)
print(text)


- [特斯拉全面叫停建廠 馬斯克擱置全球擴張野心？](https://news.cnyes.com/news/id/5682767)
- [川普告訴馬斯克：AI 耗電驚人、電動車並非人人都需要](https://technews.tw/2024/08/13/trump-thinks-not-everyone-necessarily-needs-an-ev/)
- [SpaceX將執行「人類首次」載人極地太空任務，F2Pool礦池共同創辦人任指揮官](https://www.blocktempo.com/co-founder-of-bitcoin-mining-pool-f2pool-serves-as-a-mission-commander-for-spacex/)
- [Grok-2来了，能生图识图、性能比肩GPT-4o，马斯克：发展猛如火箭](https://www.jiqizhixin.com/articles/2024-08-14-9)
- [因未回郵件「被離職」　推特前員工告馬斯克贏近2千萬賠償](https://today.line.me/tw/v2/article/kEvB050)

- [外資：Blackwell 晶片設計沒缺陷，最多遲 4~6 週](https://technews.tw/2024/08/14/blackwell-shipping-time/)
- [大摩：輝達Blackwell晶片第四季出貨時間預計不變](https://news.cnyes.com/news/id/5683440)
- [Nvidia 持股的機器人新創公司公布Q2財報 盤後股價大漲17%](https://money.udn.com/money/story/5599/8160483)
- [晶片巨頭壓力測試？Nvidia 遭競爭對手投訴的背後真相](https://technews.tw/2024/08/14/nvidia-complained-by-competitors/)
- [華為傳推 H100 競品，輝達無懼雜音、股價反飆 6%](https://finance.technews.tw/2024/08/14/nvidia-stock-price-0813/)
- [一夜之间，谷歌版GPT-4o和AI手机全上市了](https://www.jiqizhi

### See Also
- [news_query.ipynb (from GitHub)](https://colab.research.google.com/github/yorkjong/news-digest/blob/main/notebooks/news_query.ipynb)